In [1]:
import pandas as pd

In [2]:
data = pd.read_excel("First_2000_abstract_clean.xlsx", index_col=0)
data

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates
1.0,C,"Purnamasari, E; Sudarno, S",Analysis of Greenhouse Gas Reduction by Using ...,Agriculture contributes approximately 14% of g...,2018.0,10.1051/e3sconf/20187308005,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN
2.0,J,"McGee, JA",Does certified organic farming reduce greenhou...,The increasing prevalence of ecologically sust...,2015.0,10.1007/s10460-014-9543-1,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN
3.0,C,"Kononenko, K; Demidov, D",USE OF ARTIFICIAL NEURAL NETWORKS TO PREDICT G...,Agriculture has a greater impact on the enviro...,2020.0,10.22616/ERDev.2020.19.TF213,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN
4.0,J,"Chianese, DS; Rotz, CA; Richard, TL",SIMULATION OF NITROUS OXIDE EMISSIONS FROM DAI...,Farming practices can have a large impact on t...,2009.0,NaN,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN
5.0,J,"De Gryze, S; Albarracin, MV; Catala-Luque, R; ...",Modeling shows that alternative soil managemen...,Agricultural management has a significant impa...,2009.0,10.3733/ca.v063n02p84,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1996.0,J,"Edberg, S; Tigabu, M; Oden, PC",Commercial Eucalyptus Plantations with Taungya...,"The increasing demand for wood, fiber, and pul...",2022.0,10.3390/f13091395,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN
1997.0,J,"Karki, S; Adviento-Borbe, MAA; Massey, JH; Reb...",Assessing Seasonal Methane and Nitrous Oxide E...,Improved irrigation management is identified a...,2021.0,10.3390/agriculture11030261,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN
1998.0,J,"Kollah, B; Dubey, G; Parasai, P; Saha, JK; Gan...",Interactive effect of biochar size and organic...,Biochar (BC) application to agricultural soil ...,2015.0,10.1111/sum.12168,EXCLUDE,NaN,EXCLUDE,EXCLUDE,NaN,NaN
1999.0,J,"Bregaglio, S; Mongiano, G; Ferrara, RM; Ginald...",Which are the most favourable conditions for r...,No-tillage practices have a recognised benefic...,2022.0,10.1016/j.iswcr.2022.05.0032095-6339,INCLUDE,NaN,INCLUDE,INCLUDE,Review,NaN


In [4]:
data["Final decision"].value_counts()

EXCLUDE    1004
INCLUDE     996
Name: Final decision, dtype: int64

In [31]:
chatgpt_train_sample = data.sample(160, random_state=2023)
chatgpt_train_sample = chatgpt_train_sample[~chatgpt_train_sample["Abstract (AB)"].isna()]
chatgpt_train_sample.iloc[100:].shape

(59, 12)

In [12]:
import os
import openai
import time

In [1]:
openai.organization = ""#"X"
openai.api_key = "X"
openai.Model.list()

In [14]:
def chatgpt(text, temp=0.7, max_t=5):
    try: 
        output = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                  {"role": "system", "content": text},
            ],
            temperature=temp,
            max_tokens=max_t,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        openai.Completion.create(
            model="text-davinci-003",
            prompt=text,

        )
    except:
        print("Thinking...")
        time.sleep(1)
        return chatgpt(text, temp=temp, max_t=max_t)
    return output

In [15]:
template = lambda t, a, b: f"""Title: {t}
Abstract: {a}
Based on the abstract/title, {b.replace("article", "abstract")} (think step by step and answer "Yes" or "No")"""
t = chatgpt_train_sample["Article Title"].iloc[0] 
a = chatgpt_train_sample["Abstract (AB)"].iloc[0]
b = "Does the article discuss agroecological practices?"
b = b.replace("article", "abstract")
prompt = template(t, a, b)
print(prompt)

Title: Does carbon farming provide a cost-effective option to mitigate GHG emissions? Evidence from China
Abstract: In this study, we apply a whole farm bioeconomic analysis to explore the changes in land use, farm practices and on-farm greenhouse gas (GHG) emission under varying levels of agricultural greenhouse gas abatement incentives in the form of a carbon tax for a semi-arid crop-livestock farming system in China's Loess Plateau. Our results show that the optimised agricultural enterprises move towards being cropping-dominated reducing on-farm emission since livestock perform is the major source of emission. Farmers employ less oats-based and rapeseed-based rotations but more dry pea-based rotations in the optimal enterprise mix. A substantial reduction in on-farm greenhouse gas emission can be achieved at low cost with a small increase in carbon incentives. Our estimates indicate that crop-livestock farmers in China's Loess Plateau may reduce their on-farm GHG emission between 1

In [16]:
%%time
chatgpt(prompt, temp=0.7, max_t=5)["choices"][0]["message"]["content"]

Wall time: 1.94 s


'Yes'

In [32]:
%%time
b = "Does the article discuss agroecological practices?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = str(a).replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 Yes EXCLUDE
3.389830508474576 150.0 Yes EXCLUDE
5.084745762711864 360.0 Yes INCLUDE
6.779661016949152 1017.0 Yes. EXCLUDE
8.474576271186441 1231.0 No. EXCLUDE
10.169491525423728 577.0 Yes. EXCLUDE
11.864406779661017 1149.0 Yes. EXCLUDE
13.559322033898304 833.0 Yes INCLUDE
15.254237288135593 717.0 Yes INCLUDE
16.949152542372882 135.0 Yes EXCLUDE
18.64406779661017 1162.0 Yes. EXCLUDE
20.338983050847457 33.0 Yes INCLUDE
22.033898305084747 570.0 Yes INCLUDE
23.728813559322035 328.0 Yes. INCLUDE
25.423728813559322 1457.0 Yes. EXCLUDE
27.11864406779661 504.0 Yes EXCLUDE
28.8135593220339 68.0 Yes. EXCLUDE
30.508474576271187 446.0 Yes. EXCLUDE
32.20338983050848 464.0 Yes. EXCLUDE
33.898305084745765 1406.0 Yes INCLUDE
35.59322033898305 749.0 No EXCLUDE
37.28813559322034 153.0 Yes. INCLUDE
38.983050847457626 417.0 Yes INCLUDE
40.67796610169491 1029.0 Yes EXCLUDE
42.3728813559322 1976.0 Yes. INCLUDE
44.067796610169495 167.0 Yes INCLUDE
45.76271186440678 1247.0 No. EXCLUDE

In [33]:
train_b1 = chatgpt_train_sample.iloc[100:].copy()
train_b1["chatgpt"] = [x[1] for x in responses]
train_b1

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,Yes.
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes


In [34]:
train_b1.to_excel("chatgpt_responses_b1_v2.xlsx")

In [35]:
from sklearn.metrics import classification_report

In [36]:
print(classification_report(train_b1["Final decision"] == "INCLUDE", train_b1["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.9000    0.2727    0.4186        33
        True     0.5102    0.9615    0.6667        26

    accuracy                         0.5763        59
   macro avg     0.7051    0.6171    0.5426        59
weighted avg     0.7282    0.5763    0.5279        59



In [37]:
%%time
b = "Does the article focus primarily on climate change mitigation?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.index):
    t, a, label = chatgpt_train_sample.loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.shape[0], ix, o, label)

0.6289308176100629 232.0 Yes INCLUDE
1.2578616352201257 1761.0 No. INCLUDE
1.8867924528301887 792.0 Yes EXCLUDE
2.5157232704402515 405.0 Yes. INCLUDE
3.1446540880503147 1396.0 No. EXCLUDE
3.7735849056603774 433.0 No EXCLUDE
4.40251572327044 543.0 Yes. EXCLUDE
5.031446540880503 412.0 Yes INCLUDE
5.660377358490566 1397.0 Yes EXCLUDE
6.289308176100629 688.0 Yes INCLUDE
6.918238993710692 743.0 Yes. EXCLUDE
7.547169811320755 1222.0 Yes EXCLUDE
8.176100628930818 329.0 Yes EXCLUDE
8.80503144654088 1365.0 No. EXCLUDE
9.433962264150944 677.0 Yes INCLUDE
10.062893081761006 1432.0 No. The abstract discusses INCLUDE
10.69182389937107 507.0 No. EXCLUDE
11.320754716981131 1302.0 No EXCLUDE
11.949685534591195 847.0 Yes INCLUDE
12.578616352201259 1580.0 Yes EXCLUDE
13.20754716981132 1700.0 Yes EXCLUDE
13.836477987421384 920.0 Yes. INCLUDE
14.465408805031446 987.0 Yes INCLUDE
15.09433962264151 1673.0 Yes INCLUDE
15.723270440251572 312.0 Yes. INCLUDE
16.352201257861637 444.0 Yes EXCLUDE
16.9811320754717

In [38]:
train_b2 = chatgpt_train_sample.copy()
train_b2["chatgpt"] = [x[1] for x in responses]
train_b2

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
232.0,J,"Tang, K; He, CT; Ma, CB; Wang, D",Does carbon farming provide a cost-effective o...,"In this study, we apply a whole farm bioeconom...",2019.0,10.1111/1467-8489.12306,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
1761.0,J,"Allam, M; Radicetti, E; Quintarelli, V; Petros...",Influence of Organic and Mineral Fertilizers o...,The intensive use of mineral (M) fertilizers m...,2022.0,10.3390/agriculture12040464,INCLUDE,NaN,INCLUDE,INCLUDE,Review,NaN,No.
792.0,J,"Raihan, A; Said, MNM; Abdullah, SMS; Begum, RA",Climate Change Mitigation Options in the Fores...,The Intergovernmental Panel on Climate Change ...,2018.0,10.17576/jkukm-2018-si1(6)-11,EXCLUDE,NaN,EXCLUDE,EXCLUDE,NaN,NaN,Yes
405.0,J,"Wen, Y; Zang, HD; Freeman, B; Ma, QX; Chadwick...",Rye cover crop incorporation and high watertab...,Drainage and cultivation of peat soils almost ...,2019.0,10.1002/ldr.3390,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes.
1396.0,J,"Dhanda, S; Yadav, A; Yadav, DB; Chauhan, BS",Emerging Issues and Potential Opportunities in...,The rice-wheat cropping system (RWCS) is the b...,2022.0,10.3389/fpls.2022.832683,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,No.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323.0,J,"Paul, BK; Epper, CA; Tschopp, DJ; Long, CTM; T...",Crop-livestock integration provides opportunit...,CONTEXT: The Greater Mekong Subregion has been...,2022.0,10.1016/j.agsy.2021.103285,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN,Yes
394.0,J,"Cech, R; Leisch, F; Zaller, JG",Pesticide Use and Associated Greenhouse Gas Em...,The production of synthetic pesticides is ener...,2022.0,10.3390/agriculture12060879,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
1174.0,J,"Cataldo, E; Fucile, M; Mattii, GB","A Review: Soil Management, Sustainable Strateg...",Conservative and sustainable soil management i...,2021.0,10.3390/agronomy11112359,INCLUDE,INCLUDE,NaN,INCLUDE,Review,NaN,No.
1920.0,J,"De Corato, U",Agricultural waste recycling in horticultural ...,The vegetables supply chain of intensive farmi...,2020.0,10.1016/j.scitotenv.2020.139840,INCLUDE,NaN,INCLUDE,INCLUDE,Review,NaN,No.


In [39]:
train_b2.to_excel("chatgpt_responses_b2_v2.xlsx")

In [40]:
print(classification_report(train_b2["Final decision"] == "INCLUDE", train_b2["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.5357    0.1705    0.2586        88
        True     0.4427    0.8169    0.5743        71

    accuracy                         0.4591       159
   macro avg     0.4892    0.4937    0.4164       159
weighted avg     0.4942    0.4591    0.3996       159



In [41]:
%%time
b = "Does the article cover climate change adaptation?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 Yes EXCLUDE
3.389830508474576 150.0 No EXCLUDE
5.084745762711864 360.0 Yes INCLUDE
6.779661016949152 1017.0 No. The abstract covers EXCLUDE
8.474576271186441 1231.0 Yes. EXCLUDE
10.169491525423728 577.0 Yes EXCLUDE
11.864406779661017 1149.0 Yes. EXCLUDE
13.559322033898304 833.0 Yes. INCLUDE
15.254237288135593 717.0 No. INCLUDE
16.949152542372882 135.0 Yes EXCLUDE
18.64406779661017 1162.0 Yes EXCLUDE
20.338983050847457 33.0 Yes INCLUDE
22.033898305084747 570.0 Yes. INCLUDE
23.728813559322035 328.0 No INCLUDE
25.423728813559322 1457.0 Yes EXCLUDE
27.11864406779661 504.0 Yes EXCLUDE
28.8135593220339 68.0 No EXCLUDE
30.508474576271187 446.0 Yes EXCLUDE
32.20338983050848 464.0 Yes. EXCLUDE
33.898305084745765 1406.0 Yes INCLUDE
35.59322033898305 749.0 Yes EXCLUDE
37.28813559322034 153.0 Yes INCLUDE
38.983050847457626 417.0 Yes. INCLUDE
40.67796610169491 1029.0 Yes. EXCLUDE
42.3728813559322 1976.0 No. INCLUDE
44.067796610169495 167.0 No. INCLUDE
45.76271186440678 1247

In [42]:
train_b3 = chatgpt_train_sample.iloc[100:].copy()
train_b3["chatgpt"] = [x[1] for x in responses]
train_b3

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No. The abstract covers
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,Yes
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,No.
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes


In [43]:
train_b3.to_excel("chatgpt_responses_b3_v2.xlsx")

In [44]:
print(classification_report(train_b3["Final decision"] == "INCLUDE", train_b3["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.5263    0.3030    0.3846        33
        True     0.4250    0.6538    0.5152        26

    accuracy                         0.4576        59
   macro avg     0.4757    0.4784    0.4499        59
weighted avg     0.4817    0.4576    0.4421        59



In [45]:
%%time
b = "Does the article comprehensively cover climate change and environmental aspects?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 Yes. EXCLUDE
3.389830508474576 150.0 Yes EXCLUDE
5.084745762711864 360.0 Yes INCLUDE
6.779661016949152 1017.0 Yes. EXCLUDE
8.474576271186441 1231.0 Yes. EXCLUDE
10.169491525423728 577.0 Yes. EXCLUDE
11.864406779661017 1149.0 Yes EXCLUDE
13.559322033898304 833.0 Yes INCLUDE
15.254237288135593 717.0 Yes INCLUDE
16.949152542372882 135.0 Yes. EXCLUDE
18.64406779661017 1162.0 Yes EXCLUDE
20.338983050847457 33.0 Yes INCLUDE
22.033898305084747 570.0 Yes INCLUDE
23.728813559322035 328.0 Yes INCLUDE
25.423728813559322 1457.0 Yes EXCLUDE
27.11864406779661 504.0 Yes EXCLUDE
28.8135593220339 68.0 Yes EXCLUDE
30.508474576271187 446.0 Yes EXCLUDE
32.20338983050848 464.0 Yes. EXCLUDE
33.898305084745765 1406.0 Yes INCLUDE
35.59322033898305 749.0 Yes EXCLUDE
37.28813559322034 153.0 Yes INCLUDE
38.983050847457626 417.0 Yes. INCLUDE
40.67796610169491 1029.0 Yes. EXCLUDE
42.3728813559322 1976.0 No. The abstract focuses INCLUDE
44.067796610169495 167.0 Yes INCLUDE
45.76271186440678

In [46]:
train_b4 = chatgpt_train_sample.iloc[100:].copy()
train_b4["chatgpt"] = [x[1] for x in responses]
train_b4

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes.
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,Yes.
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.


In [47]:
train_b4.to_excel("chatgpt_responses_b4_v2.xlsx")

In [48]:
print(classification_report(train_b4["Final decision"] == "INCLUDE", train_b4["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.0000    0.0000    0.0000        33
        True     0.4211    0.9231    0.5783        26

    accuracy                         0.4068        59
   macro avg     0.2105    0.4615    0.2892        59
weighted avg     0.1855    0.4068    0.2548        59



In [49]:
%%time
b = "Does the study specifically evaluate greenhouse gas emissions?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.index):
    t, a, label = chatgpt_train_sample.loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.shape[0], ix, o, label)

0.6289308176100629 232.0 Yes. INCLUDE
1.2578616352201257 1761.0 No. INCLUDE
1.8867924528301887 792.0 Yes EXCLUDE
2.5157232704402515 405.0 Yes INCLUDE
3.1446540880503147 1396.0 Yes. EXCLUDE
3.7735849056603774 433.0 Yes. EXCLUDE
4.40251572327044 543.0 Yes. EXCLUDE
5.031446540880503 412.0 Yes INCLUDE
5.660377358490566 1397.0 Yes. EXCLUDE
6.289308176100629 688.0 Yes. INCLUDE
6.918238993710692 743.0 Yes EXCLUDE
7.547169811320755 1222.0 Yes. EXCLUDE
8.176100628930818 329.0 Yes. EXCLUDE
8.80503144654088 1365.0 Yes. EXCLUDE
9.433962264150944 677.0 Yes INCLUDE
10.062893081761006 1432.0 Yes. INCLUDE
10.69182389937107 507.0 Yes. EXCLUDE
11.320754716981131 1302.0 No EXCLUDE
11.949685534591195 847.0 Yes. INCLUDE
12.578616352201259 1580.0 Yes EXCLUDE
13.20754716981132 1700.0 Yes EXCLUDE
13.836477987421384 920.0 Yes INCLUDE
14.465408805031446 987.0 Yes INCLUDE
15.09433962264151 1673.0 No. INCLUDE
15.723270440251572 312.0 Yes INCLUDE
16.352201257861637 444.0 Yes EXCLUDE
16.9811320754717 1171.0 Yes. IN

In [50]:
train_b5 = chatgpt_train_sample.copy()
train_b5["chatgpt"] = [x[1] for x in responses]
train_b5

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
232.0,J,"Tang, K; He, CT; Ma, CB; Wang, D",Does carbon farming provide a cost-effective o...,"In this study, we apply a whole farm bioeconom...",2019.0,10.1111/1467-8489.12306,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
1761.0,J,"Allam, M; Radicetti, E; Quintarelli, V; Petros...",Influence of Organic and Mineral Fertilizers o...,The intensive use of mineral (M) fertilizers m...,2022.0,10.3390/agriculture12040464,INCLUDE,NaN,INCLUDE,INCLUDE,Review,NaN,No.
792.0,J,"Raihan, A; Said, MNM; Abdullah, SMS; Begum, RA",Climate Change Mitigation Options in the Fores...,The Intergovernmental Panel on Climate Change ...,2018.0,10.17576/jkukm-2018-si1(6)-11,EXCLUDE,NaN,EXCLUDE,EXCLUDE,NaN,NaN,Yes
405.0,J,"Wen, Y; Zang, HD; Freeman, B; Ma, QX; Chadwick...",Rye cover crop incorporation and high watertab...,Drainage and cultivation of peat soils almost ...,2019.0,10.1002/ldr.3390,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1396.0,J,"Dhanda, S; Yadav, A; Yadav, DB; Chauhan, BS",Emerging Issues and Potential Opportunities in...,The rice-wheat cropping system (RWCS) is the b...,2022.0,10.3389/fpls.2022.832683,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323.0,J,"Paul, BK; Epper, CA; Tschopp, DJ; Long, CTM; T...",Crop-livestock integration provides opportunit...,CONTEXT: The Greater Mekong Subregion has been...,2022.0,10.1016/j.agsy.2021.103285,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN,Yes.
394.0,J,"Cech, R; Leisch, F; Zaller, JG",Pesticide Use and Associated Greenhouse Gas Em...,The production of synthetic pesticides is ener...,2022.0,10.3390/agriculture12060879,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes.
1174.0,J,"Cataldo, E; Fucile, M; Mattii, GB","A Review: Soil Management, Sustainable Strateg...",Conservative and sustainable soil management i...,2021.0,10.3390/agronomy11112359,INCLUDE,INCLUDE,NaN,INCLUDE,Review,NaN,Yes
1920.0,J,"De Corato, U",Agricultural waste recycling in horticultural ...,The vegetables supply chain of intensive farmi...,2020.0,10.1016/j.scitotenv.2020.139840,INCLUDE,NaN,INCLUDE,INCLUDE,Review,NaN,Yes


In [51]:
train_b5.to_excel("chatgpt_responses_b5_v2.xlsx")

In [52]:
print(classification_report(train_b5["Final decision"] == "INCLUDE", train_b5["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.5455    0.0682    0.1212        88
        True     0.4459    0.9296    0.6027        71

    accuracy                         0.4528       159
   macro avg     0.4957    0.4989    0.3620       159
weighted avg     0.5010    0.4528    0.3362       159



In [53]:
%%time
b = "Does the article assess agroecological practices' impact on climate change?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 Yes EXCLUDE
3.389830508474576 150.0 No EXCLUDE
5.084745762711864 360.0 Yes INCLUDE
6.779661016949152 1017.0 No EXCLUDE
8.474576271186441 1231.0 No EXCLUDE
10.169491525423728 577.0 Yes EXCLUDE
11.864406779661017 1149.0 No. EXCLUDE
13.559322033898304 833.0 Yes INCLUDE
15.254237288135593 717.0 Yes INCLUDE
16.949152542372882 135.0 Yes. EXCLUDE
18.64406779661017 1162.0 Yes EXCLUDE
20.338983050847457 33.0 Yes. INCLUDE
22.033898305084747 570.0 Yes INCLUDE
23.728813559322035 328.0 Yes. INCLUDE
25.423728813559322 1457.0 Yes. EXCLUDE
27.11864406779661 504.0 Yes EXCLUDE
28.8135593220339 68.0 No. EXCLUDE
30.508474576271187 446.0 Yes EXCLUDE
32.20338983050848 464.0 The abstract mentions nitrous EXCLUDE
33.898305084745765 1406.0 Yes. INCLUDE
35.59322033898305 749.0 Yes EXCLUDE
37.28813559322034 153.0 Yes. INCLUDE
38.983050847457626 417.0 Yes INCLUDE
40.67796610169491 1029.0 Yes. EXCLUDE
42.3728813559322 1976.0 Yes. INCLUDE
44.067796610169495 167.0 Yes. INCLUDE
45.76271186440

In [54]:
train_b6 = chatgpt_train_sample.iloc[100:].copy()
train_b6["chatgpt"] = [x[1] for x in responses]
train_b6

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,Yes
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.


In [55]:
train_b6.to_excel("chatgpt_responses_b6_v2.xlsx")

In [56]:
print(classification_report(train_b6["Final decision"] == "INCLUDE", train_b6["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     1.0000    0.3030    0.4651        33
        True     0.5306    1.0000    0.6933        26

    accuracy                         0.6102        59
   macro avg     0.7653    0.6515    0.5792        59
weighted avg     0.7932    0.6102    0.5657        59



In [57]:
%%time
b = "Does the article target stakeholders?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.index):
    t, a, label = chatgpt_train_sample.loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.shape[0], ix, o, label)

0.6289308176100629 232.0 Yes. INCLUDE
1.2578616352201257 1761.0 Yes INCLUDE
1.8867924528301887 792.0 Yes. EXCLUDE
2.5157232704402515 405.0 Yes. INCLUDE
3.1446540880503147 1396.0 Yes EXCLUDE
3.7735849056603774 433.0 Yes EXCLUDE
4.40251572327044 543.0 Yes. EXCLUDE
5.031446540880503 412.0 Yes. INCLUDE
5.660377358490566 1397.0 Yes EXCLUDE
6.289308176100629 688.0 Yes INCLUDE
6.918238993710692 743.0 Yes. EXCLUDE
7.547169811320755 1222.0 Yes. EXCLUDE
8.176100628930818 329.0 Yes. EXCLUDE
8.80503144654088 1365.0 Yes. EXCLUDE
9.433962264150944 677.0 Yes INCLUDE
10.062893081761006 1432.0 Yes INCLUDE
10.69182389937107 507.0 Yes EXCLUDE
11.320754716981131 1302.0 Yes EXCLUDE
11.949685534591195 847.0 Yes. INCLUDE
12.578616352201259 1580.0 Yes EXCLUDE
13.20754716981132 1700.0 Yes. EXCLUDE
13.836477987421384 920.0 Yes INCLUDE
14.465408805031446 987.0 Yes INCLUDE
15.09433962264151 1673.0 Yes. INCLUDE
15.723270440251572 312.0 Yes INCLUDE
16.352201257861637 444.0 Yes EXCLUDE
16.9811320754717 1171.0 Yes IN

In [58]:
train_b7 = chatgpt_train_sample.copy()
train_b7["chatgpt"] = [x[1] for x in responses]
train_b7

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
232.0,J,"Tang, K; He, CT; Ma, CB; Wang, D",Does carbon farming provide a cost-effective o...,"In this study, we apply a whole farm bioeconom...",2019.0,10.1111/1467-8489.12306,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
1761.0,J,"Allam, M; Radicetti, E; Quintarelli, V; Petros...",Influence of Organic and Mineral Fertilizers o...,The intensive use of mineral (M) fertilizers m...,2022.0,10.3390/agriculture12040464,INCLUDE,NaN,INCLUDE,INCLUDE,Review,NaN,Yes
792.0,J,"Raihan, A; Said, MNM; Abdullah, SMS; Begum, RA",Climate Change Mitigation Options in the Fores...,The Intergovernmental Panel on Climate Change ...,2018.0,10.17576/jkukm-2018-si1(6)-11,EXCLUDE,NaN,EXCLUDE,EXCLUDE,NaN,NaN,Yes.
405.0,J,"Wen, Y; Zang, HD; Freeman, B; Ma, QX; Chadwick...",Rye cover crop incorporation and high watertab...,Drainage and cultivation of peat soils almost ...,2019.0,10.1002/ldr.3390,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes.
1396.0,J,"Dhanda, S; Yadav, A; Yadav, DB; Chauhan, BS",Emerging Issues and Potential Opportunities in...,The rice-wheat cropping system (RWCS) is the b...,2022.0,10.3389/fpls.2022.832683,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323.0,J,"Paul, BK; Epper, CA; Tschopp, DJ; Long, CTM; T...",Crop-livestock integration provides opportunit...,CONTEXT: The Greater Mekong Subregion has been...,2022.0,10.1016/j.agsy.2021.103285,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN,Yes
394.0,J,"Cech, R; Leisch, F; Zaller, JG",Pesticide Use and Associated Greenhouse Gas Em...,The production of synthetic pesticides is ener...,2022.0,10.3390/agriculture12060879,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
1174.0,J,"Cataldo, E; Fucile, M; Mattii, GB","A Review: Soil Management, Sustainable Strateg...",Conservative and sustainable soil management i...,2021.0,10.3390/agronomy11112359,INCLUDE,INCLUDE,NaN,INCLUDE,Review,NaN,Yes.
1920.0,J,"De Corato, U",Agricultural waste recycling in horticultural ...,The vegetables supply chain of intensive farmi...,2020.0,10.1016/j.scitotenv.2020.139840,INCLUDE,NaN,INCLUDE,INCLUDE,Review,NaN,Yes


In [59]:
train_b7.to_excel("chatgpt_responses_b7_v2.xlsx")

In [2]:
print(classification_report(train_b7["Final decision"] == "INCLUDE", train_b7["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

In [61]:
# %%time
b = "Does the article address limitations, challenges, and potential risks?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 No. EXCLUDE
3.389830508474576 150.0 Yes EXCLUDE
5.084745762711864 360.0 No INCLUDE
6.779661016949152 1017.0 Yes. EXCLUDE
8.474576271186441 1231.0 Yes. EXCLUDE
10.169491525423728 577.0 Yes. The abstract discusses EXCLUDE
11.864406779661017 1149.0 Yes. The abstract mentions EXCLUDE
13.559322033898304 833.0 No. INCLUDE
15.254237288135593 717.0 Yes. The abstract mentions INCLUDE
16.949152542372882 135.0 Yes. The abstract mentions EXCLUDE
18.64406779661017 1162.0 No. The abstract mainly EXCLUDE
20.338983050847457 33.0 No INCLUDE
22.033898305084747 570.0 Yes INCLUDE
23.728813559322035 328.0 Yes. The abstract mentions INCLUDE
25.423728813559322 1457.0 Yes EXCLUDE
27.11864406779661 504.0 No EXCLUDE
28.8135593220339 68.0 Yes EXCLUDE
30.508474576271187 446.0 Yes EXCLUDE
32.20338983050848 464.0 No EXCLUDE
33.898305084745765 1406.0 Yes. The abstract mentions INCLUDE
35.59322033898305 749.0 Yes. EXCLUDE
37.28813559322034 153.0 Yes. INCLUDE
38.983050847457626 417.0 No. INCLU

In [62]:
train_b8 = chatgpt_train_sample.iloc[100:].copy()
train_b8["chatgpt"] = [x[1] for x in responses]
train_b8

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,No.
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,No
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,Yes. The abstract discusses
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes. The abstract mentions
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,No.
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes. The abstract mentions
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes. The abstract mentions


In [63]:
train_b8.to_excel("D:/github/abstract-prediction/chatgpt_responses_b8_v2.xlsx")

In [64]:
print(classification_report(train_b8["Final decision"] == "INCLUDE", train_b8["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.4583    0.3333    0.3860        33
        True     0.3714    0.5000    0.4262        26

    accuracy                         0.4068        59
   macro avg     0.4149    0.4167    0.4061        59
weighted avg     0.4200    0.4068    0.4037        59



In [65]:
%%time
b = "Does the article address policy implications?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 Yes EXCLUDE
3.389830508474576 150.0 No. EXCLUDE
5.084745762711864 360.0 Yes INCLUDE
6.779661016949152 1017.0 Yes EXCLUDE
8.474576271186441 1231.0 Yes EXCLUDE
10.169491525423728 577.0 Yes EXCLUDE
11.864406779661017 1149.0 Yes. EXCLUDE
13.559322033898304 833.0 Yes. INCLUDE
15.254237288135593 717.0 Yes INCLUDE
16.949152542372882 135.0 Yes EXCLUDE
18.64406779661017 1162.0 No. EXCLUDE
20.338983050847457 33.0 No INCLUDE
22.033898305084747 570.0 No INCLUDE
23.728813559322035 328.0 No INCLUDE
25.423728813559322 1457.0 Yes EXCLUDE
27.11864406779661 504.0 Yes EXCLUDE
28.8135593220339 68.0 Yes EXCLUDE
30.508474576271187 446.0 Yes EXCLUDE
32.20338983050848 464.0 No EXCLUDE
33.898305084745765 1406.0 Yes INCLUDE
35.59322033898305 749.0 No. EXCLUDE
37.28813559322034 153.0 Yes INCLUDE
38.983050847457626 417.0 Yes INCLUDE
40.67796610169491 1029.0 Yes EXCLUDE
42.3728813559322 1976.0 No. INCLUDE
44.067796610169495 167.0 Yes INCLUDE
45.76271186440678 1247.0 Yes EXCLUDE
47.45762711

In [66]:
train_b9 = chatgpt_train_sample.iloc[100:].copy()
train_b9["chatgpt"] = [x[1] for x in responses]
train_b9

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,Yes
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes.
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes


In [67]:
train_b9.to_excel("D:/github/abstract-prediction/chatgpt_responses_b9_v2.xlsx")

In [68]:
print(classification_report(train_b9["Final decision"] == "INCLUDE", train_b9["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.5294    0.2727    0.3600        33
        True     0.4286    0.6923    0.5294        26

    accuracy                         0.4576        59
   macro avg     0.4790    0.4825    0.4447        59
weighted avg     0.4850    0.4576    0.4347        59



In [69]:
%%time
b = "Does the article have a specific geographic focus?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 Yes EXCLUDE
3.389830508474576 150.0 No. EXCLUDE
5.084745762711864 360.0 No. INCLUDE
6.779661016949152 1017.0 Yes. The abstract has EXCLUDE
8.474576271186441 1231.0 No EXCLUDE
10.169491525423728 577.0 Yes. The abstract specifically EXCLUDE
11.864406779661017 1149.0 No EXCLUDE
13.559322033898304 833.0 Yes INCLUDE
15.254237288135593 717.0 Yes. INCLUDE
16.949152542372882 135.0 Yes EXCLUDE
18.64406779661017 1162.0 No EXCLUDE
20.338983050847457 33.0 No INCLUDE
22.033898305084747 570.0 No INCLUDE
23.728813559322035 328.0 No INCLUDE
25.423728813559322 1457.0 Yes. EXCLUDE
27.11864406779661 504.0 Yes EXCLUDE
28.8135593220339 68.0 Yes EXCLUDE
30.508474576271187 446.0 Yes. The abstract mentions EXCLUDE
32.20338983050848 464.0 No EXCLUDE
33.898305084745765 1406.0 No INCLUDE
35.59322033898305 749.0 No EXCLUDE
37.28813559322034 153.0 Yes INCLUDE
38.983050847457626 417.0 Yes INCLUDE
40.67796610169491 1029.0 Yes. The abstract has EXCLUDE
42.3728813559322 1976.0 No INCLUDE
44.06

In [70]:
train_b10 = chatgpt_train_sample.iloc[100:].copy()
train_b10["chatgpt"] = [x[1] for x in responses]
train_b10

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,No.
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes. The abstract has
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,Yes. The abstract specifically
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,Yes


In [71]:
train_b10.to_excel("D:/github/abstract-prediction/chatgpt_responses_b10_v2.xlsx")

In [72]:
print(classification_report(train_b10["Final decision"] == "INCLUDE", train_b10["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.4667    0.2121    0.2917        33
        True     0.4091    0.6923    0.5143        26

    accuracy                         0.4237        59
   macro avg     0.4379    0.4522    0.4030        59
weighted avg     0.4413    0.4237    0.3898        59



In [73]:
%%time
b = "Does the article discuss the role of agroecology in enhancing climate resilience in agricultural systems?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 Yes EXCLUDE
3.389830508474576 150.0 No. EXCLUDE
5.084745762711864 360.0 Yes. INCLUDE
6.779661016949152 1017.0 No EXCLUDE
8.474576271186441 1231.0 No. EXCLUDE
10.169491525423728 577.0 No. EXCLUDE
11.864406779661017 1149.0 No. The abstract discusses EXCLUDE
13.559322033898304 833.0 Yes. INCLUDE
15.254237288135593 717.0 No INCLUDE
16.949152542372882 135.0 No. EXCLUDE
18.64406779661017 1162.0 No EXCLUDE
20.338983050847457 33.0 Yes INCLUDE
22.033898305084747 570.0 Yes. The abstract discusses INCLUDE
23.728813559322035 328.0 Yes INCLUDE
25.423728813559322 1457.0 No. EXCLUDE
27.11864406779661 504.0 Yes. EXCLUDE
28.8135593220339 68.0 No EXCLUDE
30.508474576271187 446.0 No. EXCLUDE
32.20338983050848 464.0 No. EXCLUDE
33.898305084745765 1406.0 Yes INCLUDE
35.59322033898305 749.0 No. The abstract does EXCLUDE
37.28813559322034 153.0 No. The abstract discusses INCLUDE
38.983050847457626 417.0 No. INCLUDE
40.67796610169491 1029.0 No. EXCLUDE
42.3728813559322 1976.0 No. The 

In [74]:
train_b11 = chatgpt_train_sample.iloc[100:].copy()
train_b11["chatgpt"] = [x[1] for x in responses]
train_b11

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,Yes
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes.
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,No.
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No. The abstract discusses
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,No
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.


In [75]:
train_b11.to_excel("D:/github/abstract-prediction/chatgpt_responses_b11_v2.xlsx")

In [76]:
print(classification_report(train_b11["Final decision"] == "INCLUDE", train_b11["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.7273    0.7273    0.7273        33
        True     0.6538    0.6538    0.6538        26

    accuracy                         0.6949        59
   macro avg     0.6906    0.6906    0.6906        59
weighted avg     0.6949    0.6949    0.6949        59



In [77]:
%%time
b = "Does the article assess the impact of agroecology on nitrogen dynamics?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 No. EXCLUDE
3.389830508474576 150.0 No. EXCLUDE
5.084745762711864 360.0 Yes INCLUDE
6.779661016949152 1017.0 No. EXCLUDE
8.474576271186441 1231.0 No. EXCLUDE
10.169491525423728 577.0 No. EXCLUDE
11.864406779661017 1149.0 No EXCLUDE
13.559322033898304 833.0 Yes. INCLUDE
15.254237288135593 717.0 Yes. INCLUDE
16.949152542372882 135.0 No. EXCLUDE
18.64406779661017 1162.0 No. EXCLUDE
20.338983050847457 33.0 Yes. INCLUDE
22.033898305084747 570.0 Yes INCLUDE
23.728813559322035 328.0 Yes INCLUDE
25.423728813559322 1457.0 No. EXCLUDE
27.11864406779661 504.0 No. EXCLUDE
28.8135593220339 68.0 No. The abstract discusses EXCLUDE
30.508474576271187 446.0 No. EXCLUDE
32.20338983050848 464.0 No. EXCLUDE
33.898305084745765 1406.0 Yes. INCLUDE
35.59322033898305 749.0 No. EXCLUDE
37.28813559322034 153.0 No. The abstract does INCLUDE
38.983050847457626 417.0 Yes INCLUDE
40.67796610169491 1029.0 No. EXCLUDE
42.3728813559322 1976.0 Yes. INCLUDE
44.067796610169495 167.0 No. INCLUDE
4

In [78]:
train_b12 = chatgpt_train_sample.iloc[100:].copy()
train_b12["chatgpt"] = [x[1] for x in responses]
train_b12

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,No.
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,No.
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.


In [79]:
train_b12.to_excel("D:/github/abstract-prediction/chatgpt_responses_b12_v2.xlsx")

In [80]:
print(classification_report(train_b12["Final decision"] == "INCLUDE", train_b12["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.7273    0.9697    0.8312        33
        True     0.9333    0.5385    0.6829        26

    accuracy                         0.7797        59
   macro avg     0.8303    0.7541    0.7570        59
weighted avg     0.8181    0.7797    0.7658        59



In [81]:
%%time
b = "Does the article discuss the impact of methane (CH4) emissions?"
b = b.replace("article", "abstract")
responses = []
for i, ix in enumerate(chatgpt_train_sample.iloc[100:].index):
    t, a, label = chatgpt_train_sample.iloc[100:].loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(template(t, a, b), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/chatgpt_train_sample.iloc[100:].shape[0], ix, o, label)

1.694915254237288 1603.0 No. EXCLUDE
3.389830508474576 150.0 No. EXCLUDE
5.084745762711864 360.0 Yes INCLUDE
6.779661016949152 1017.0 No. EXCLUDE
8.474576271186441 1231.0 No. EXCLUDE
10.169491525423728 577.0 No. EXCLUDE
11.864406779661017 1149.0 The abstract does not mention EXCLUDE
13.559322033898304 833.0 No. INCLUDE
15.254237288135593 717.0 Yes. INCLUDE
16.949152542372882 135.0 No. EXCLUDE
18.64406779661017 1162.0 Yes. The abstract mentions EXCLUDE
20.338983050847457 33.0 Yes INCLUDE
22.033898305084747 570.0 No. INCLUDE
23.728813559322035 328.0 Yes INCLUDE
25.423728813559322 1457.0 Yes. EXCLUDE
27.11864406779661 504.0 No. EXCLUDE
28.8135593220339 68.0 No. EXCLUDE
30.508474576271187 446.0 Yes. The abstract mentions EXCLUDE
32.20338983050848 464.0 Yes EXCLUDE
33.898305084745765 1406.0 Yes INCLUDE
35.59322033898305 749.0 Yes EXCLUDE
37.28813559322034 153.0 Yes INCLUDE
38.983050847457626 417.0 No. INCLUDE
40.67796610169491 1029.0 Yes. EXCLUDE
42.3728813559322 1976.0 No. INCLUDE
44.06779

In [82]:
train_b13 = chatgpt_train_sample.iloc[100:].copy()
train_b13["chatgpt"] = [x[1] for x in responses]
train_b13

,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
1603.0,J,"Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH",Carbon sequestration and vegetation properties...,Exclosure management is becoming a common appr...,2020.0,10.1016/j.jnc.2020.125856,NaN,EXCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,No.
150.0,J,"Kim, S; Dale, BE; Drzal, LT; Misra, M",Life cycle assessment of kenaf fiber reinforce...,This study investigates the environmental perf...,2008.0,10.1166/jbmb.2008.207,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
360.0,J,"Wen, Y; Zang, HD; Ma, QX; Freeman, B; Chadwick...",Impact of water table levels and winter cover ...,Drainage and cultivation have turned peatlands...,2020.0,10.1016/j.scitotenv.2019.135130,NaN,INCLUDE,INCLUDE,INCLUDE,NaN,NaN,Yes
1017.0,J,"Desplechin, E",The Brazilian sugarcane ethanol industry's app...,The Brazilian sugarcane industry is well posit...,2010.0,NaN,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
1231.0,J,"Davies, WJ; Ward, SE; Wilson, A",Can crop science really help us to produce mor...,This paper reviews recent developments in crop...,2020.0,10.15302/J-FASE-2019299,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.
577.0,J,"Morand, H; Thomassin, PJ",Changes in Quebec cropping practices in respon...,This paper investigates the response of the Qu...,2005.0,10.1111/j.1744-7976.2005.00027.x,EXCLUDE,EXCLUDE,INCLUDE,EXCLUDE,NaN,NaN,No.
1149.0,J,"Veron, SR; Jobbagy, EG; Di Bella, CM; Paruelo,...",Assessing the potential of wildfires as a sust...,As the environmental and economic consequences...,2012.0,10.1111/j.1757-1707.2012.01181.x,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,The abstract does not mention
833.0,J,"Yao, ZY; Zhang, DB; Yao, PW; Zhao, N; Liu, N; ...",Coupling life-cycle assessment and the RothC m...,Reducing the carbon footprint (CF) of crop pro...,2017.0,10.1016/j.scitotenv.2017.07.028,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,No.
717.0,J,"Qin, YM; Liu, SW; Guo, YQ; Liu, QH; Zou, JW",Methane and nitrous oxide emissions from organ...,To evaluate the impacts of organic cropping sy...,2010.0,10.1007/s00374-010-0493-5,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,Yes.
135.0,J,"Venkat, K",Comparison of Twelve Organic and Conventional ...,Given the growing importance of organic food p...,2012.0,10.1080/10440046.2012.672378,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,No.


In [83]:
train_b13.to_excel("D:/github/abstract-prediction/chatgpt_responses_b13_v2.xlsx")

In [84]:
print(classification_report(train_b13["Final decision"] == "INCLUDE", train_b13["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))

              precision    recall  f1-score   support

       False     0.5758    0.5758    0.5758        33
        True     0.4615    0.4615    0.4615        26

    accuracy                         0.5254        59
   macro avg     0.5186    0.5186    0.5186        59
weighted avg     0.5254    0.5254    0.5254        59



In [28]:
columns = "Article Title  Abstract (AB)".split("  ")
recols = {"Article Title": "t", "Abstract (AB)": "ab"}
data[columns].rename(columns=recols).to_excel("2000_abstract.xlsx")